<a href="https://colab.research.google.com/github/ahpoise/ahpoise/blob/main/202509_practical_ai_development_w1_d3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Setup better response formatting (adds line wrap)
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
pip install --quiet openai tiktoken

In [ ]:
import os
from openai import OpenAI
from google.colab import userdata

client = OpenAI(
    api_key=userdata.get("OPENAI_API_KEY"),
)

model="gpt-4o-mini"
# model="o4-mini"

# user_input = "Blue"
user_input = "Once upon a time, a world existed where people"

# Tokenization

[OpenaAI Tiktoken](https://github.com/openai/tiktoken)\
[PyPI package](https://pypi.org/project/tiktoken/)

In [ ]:
import tiktoken

# Using the tokenization specifically used for model gpt-4o
encoder = tiktoken.encoding_for_model(model)
encodedText = encoder.encode(user_input)

print("encoded text:", encodedText)
print("decoded text:", encoder.decode(encodedText))
print("number of tokens:", len(encodedText))
print('\n')

for tokenId in encodedText:
  print(f"{tokenId} - {encoder.decode([tokenId])}")

encoded text: [18049, 7557, 261, 1058, 11, 261, 2375, 49030, 1919, 1665]
decoded text: Once upon a time, a world existed where people
number of tokens: 10


18049 - Once
7557 -  upon
261 -  a
1058 -  time
11 - ,
261 -  a
2375 -  world
49030 -  existed
1919 -  where
1665 -  people


# Embedding

In [ ]:
response = client.embeddings.create(
    input=user_input,
    model="text-embedding-3-large"
)

embedding = response.data[0].embedding


print("Input: ", user_input)
print('\n')
print(embedding)
print('\n')
print("Length: ", len(embedding))

# Request Parameters

Some also usable in OpenAI playground

In [ ]:
response = client.responses.create(
    model=model,
    input=user_input,
)

response.output[0].content[0].text

'Once upon a time, a world existed where people communicated not with words, but through colors and sounds. In this enchanting realm, each emotion emitted a distinct hue, and every thought resonated with a unique melody. Joy shimmered in vibrant yellows, sadness dripped in deep blues, while anger crackled with fiery reds.\n\nThe inhabitants, known as the Chromalites, wore garments woven from the threads of their emotions. Each day, they gathered in the Luminary Square, where the vibrant display of color and sound created a symphony unlike any other. In this space, disputes were resolved by blending colors, forming new shades that represented compromise and understanding.\n\nHowever, one day, a mysterious figure cloaked in grey arrived. This stranger, known as the Dull, had the ability to sap colors and silence melodies, leaving the Chromalites in despair. A once bright square turned muted, and the joyous expressions faded as fear spread through the air like a fog.\n\nDetermined to rest

## Max Output Tokens

Default: unlimited, up to the model’s maximum context window.
For reasoning models, it’s recommended to reserve ~25k tokens for hidden reasoning until you understand how many reasoning tokens your use case consumes.



In [ ]:
def generateWithMaxTokens(maxTokens):
    response = client.responses.create(
        model=model,
        input=user_input,
        max_output_tokens=maxTokens
    )
    return response.output[0].content[0].text

for maxTokens in [16, 50, 100]:
    print("-" * 80)
    print(f"Max Tokens: {maxTokens}")
    print(f"Generated Text: {generateWithMaxTokens(maxTokens)}")

--------------------------------------------------------------------------------
Max Tokens: 16
Generated Text: Once upon a time, a world existed where people communicated not through words, but
--------------------------------------------------------------------------------
Max Tokens: 50
Generated Text: Once upon a time, a world existed where people lived in harmony with nature. In this land, towering trees whispered ancient secrets, rivers sparkled like diamonds, and mountains stood as wise guardians of the valleys below. The sky was painted in hues of
--------------------------------------------------------------------------------
Max Tokens: 100
Generated Text: Once upon a time, a world existed where people communicated not with words, but with colors. Each emotion they felt was expressed as a vibrant hue, painting the air around them with shimmering shades. Joy sparkled in golden yellows, sorrow flowed like deep blues, and love radiated in soft pinks.

In this world lived a young

## Text Format

Default: text

Configuration options for a text response from the model. Can be plain text or structured JSON data.

In [ ]:
def generateWithFormat(format):
    response = client.responses.create(
        model=model,
        input=user_input,
        text=format,
        max_output_tokens=50
    )
    return response.output[0].content[0].text

for format in [
    {"format": {"type": "text"}},
    # {"format": {"type": "json_object"}},
    {
      "format": {
        "type": "json_schema",
        "name": "my_schema",
        "schema": {
          "type": "object",
          "properties": {
              "answer": {"type": "string"}
          },
          "required": ["answer"],
          "additionalProperties": False
        }
      }
    }, ]:
    print("-" * 80)
    print(f"Format: {format}")
    print(f"Generated Text: {generateWithFormat(format)}")

--------------------------------------------------------------------------------
Format: {'format': {'type': 'text'}}
Generated Text: lived in harmony with nature and each other. In this enchanting realm, lush forests stretched endlessly, vibrant flowers bloomed in every color imaginable, and rivers sparkled like diamonds under the sun. The people of this world had a profound connection to the
--------------------------------------------------------------------------------
Format: {'format': {'type': 'json_schema', 'name': 'my_schema', 'schema': {'type': 'object', 'properties': {'answer': {'type': 'string'}}, 'required': ['answer'], 'additionalProperties': False}}}
Generated Text: {"answer":"had the ability to communicate with nature. The trees whispered secrets in the wind, rivers sang melodies, and animals shared their wisdom with the villagers. This unique connection allowed the people to live in harmony with their surroundings.\n\nIn this


## Temperature

Default: 1

Higher values will make the output more random, while lower values will make it more focused and deterministic.

In [ ]:
def generateWithTemp(temp):
    response = client.responses.create(
        model=model,
        input=user_input,
        max_output_tokens=50,
        temperature=temp
    )
    return response.output[0].content[0].text

for temp in [0, 0.5, 1, 1.5, 2]:
    print("-" * 80)
    print(f"Temperature: {temp}")
    print(f"Generated Text: {generateWithTemp(temp)}")

--------------------------------------------------------------------------------
Temperature: 0
Generated Text: Once upon a time, a world existed where people lived in harmony with nature, surrounded by lush forests, sparkling rivers, and towering mountains. In this land, every individual possessed a unique gift that connected them to the elements—some could summon the wind
--------------------------------------------------------------------------------
Temperature: 0.5
Generated Text: Once upon a time, a world existed where people lived in harmony with nature. In this vibrant land, lush forests thrived alongside crystal-clear rivers, and mountains stood tall, their peaks kissed by the clouds. The inhabitants, known as the Eldori
--------------------------------------------------------------------------------
Temperature: 1
Generated Text: Once upon a time, a world existed where people lived in harmony with nature. In this enchanting land, towering trees whispered secrets to each other

## Top P

Default: 1

The model considers the results of the tokens with top_p probability mass. So 0.1 means only the tokens comprising the top 10% probability mass are considered.
Not recommended to use in combination with temperature.

In [ ]:
def generateWithFP(selected_top_p):
    response = client.responses.create(
        model=model,
        input=user_input,
        max_output_tokens=50,
        top_p=selected_top_p
    )
    return response.output[0].content[0].text

for selected_top_p in [0.1, 0.5, 1]:
  print("-" * 80)
  print(f"Top-P: {selected_top_p}")
  print(f"Generated Text: {generateWithFP(selected_top_p)}")

--------------------------------------------------------------------------------
Top-P: 0.1
Generated Text: Once upon a time, a world existed where people lived in harmony with nature, surrounded by lush forests, sparkling rivers, and towering mountains. In this land, every village was connected by winding paths that glimmered with the magic of the earth.
--------------------------------------------------------------------------------
Top-P: 0.5
Generated Text: Once upon a time, a world existed where people lived in harmony with nature. In this land, towering trees whispered secrets to the wind, and rivers sparkled like diamonds under the sun. The inhabitants, known as the Eldara, had a unique
--------------------------------------------------------------------------------
Top-P: 1
Generated Text: Once upon a time, a world existed where people communicated through music rather than words. Each person's voice was a unique melody, and conversations were vibrant symphonies. Joyful notes d

# Request Parameters - new model(s)

Some also usable in OpenAI playground

## Reasoning Effort

Default: medium

Selects reasoning effort. Reducing reasoning effort can result in faster responses and fewer tokens used on reasoning in a response.

In [ ]:
def customReasoning(selected_effort):
    response = client.responses.create(
        model=model,
        input="How much wood would a woodchuck chuck?",
        reasoning={
          "effort": selected_effort,
        },
    )
    return response.output_text

for selected_effort in ["low", "medium", "high"]:
  print("-" * 80)
  print(f"Effort: {selected_effort}")
  print(f"Generated Text: {customReasoning(selected_effort)}")

--------------------------------------------------------------------------------
Effort: low
Generated Text: A woodchuck would chuck as much wood as a woodchuck could chuck if a woodchuck could chuck wood.  

(Joking aside, a 1988 estimate by wildlife biologist Richard Thomas put that “as much” at about 700 pounds, based on the volume of dirt a woodchuck moves when digging its burrow.)
--------------------------------------------------------------------------------
Effort: medium
Generated Text: The traditional tongue-twister answer:
“A woodchuck would chuck as much wood as a woodchuck could chuck if a woodchuck could chuck wood.”

For a (semi-serious) “scientific” estimate:
• In 1988 wildlife biologist Richard Thomas calculated that, by volume, the amount of earth a woodchuck moves when digging its burrow would equate to roughly 700 lb (≈320 kg) of wood.  
• So, if a woodchuck really did “chuck” wood instead of dirt, it might manage on the order of 700 pounds in a day.
---------------

## Tools

An array of tools the model may call while generating a response.

In [ ]:
response = client.responses.create(
    model=model,
    tools=[{ "type": "web_search_preview" }],
    input="What was a positive news story from today?",
)


print(f"Effort: {response.output_text}")
# print("-" * 80)
# print(f"Generated Text: {customReasoning(selected_effort)}")


Effort: **Introduction**  
On September 25, 2025, NASA and the Indian Space Research Organisation (ISRO) unveiled the first radar images from their joint Earth‐mapping satellite, NISAR (NASA–ISRO Synthetic Aperture Radar). These inaugural images—showcasing the rugged coastline of Maine and the patchwork farmland of Minnesota—mark a pivotal milestone in a $1.3 billion mission designed to monitor minute changes in Earth’s surface for applications ranging from disaster response to ecosystem management ([apnews.com](https://apnews.com/article/73f610a8e03fa544db8826b170ce0fcc?utm_source=openai)). The high‐resolution imagery was acquired nearly two months after NISAR’s launch on July 30, 2025, into a near‐polar Sun‐synchronous orbit at an altitude of approximately 464 miles (747 km) ([apnews.com](https://apnews.com/article/73f610a8e03fa544db8826b170ce0fcc?utm_source=openai)).

---

**Mission Objectives and Technical Specifications**  
NISAR is the first spaceborne mission to carry dual‐frequ